In [1]:
import pandas as pd
import regex as re
from collections import Counter
import json
import time
import requests
import collections
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import glob
%set_env folder = X:\5_Research\Paul\dsi_origins\

env: folder=X:\5_Research\Paul\dsi_origins\


In [2]:
path = %env folder


In [ ]:
df_wildsi = pd.read_csv(path + R'processed_data\wildsi_country_idselect.csv', index_col = 0, keep_default_na=False, na_values=[''], dtype={'id_select': 'str'})

In [ ]:
df_scholarly = pd.read_csv(path+R'processed_data\scholarly_aff_group.csv', index_col = 0, keep_default_na=False, na_values=[''], dtype={'id_select': 'str'})

## 1. Construct coo-cor connections

In [ ]:
len(set(df_wildsi['id_select'].dropna()))

In [72]:
len(set(df_scholarly['id_select']))

94772

In [14]:
df_scholarly_country = pd.merge(df_scholarly, df_wildsi[['id_select', 'ISO_ori', 'GROUP_ori']].drop_duplicates())

In [15]:
len(set(df_scholarly_country['id_select']))

94772

In [18]:
df_scholarly_country.to_csv(path+R'processed_data\coo_coor_linkages.csv')

## 2. Build extractivism variable

In [6]:
df_scholarly_country = pd.read_csv(path+R'processed_data\coo_coor_linkages.csv', index_col = 0, keep_default_na=False, na_values=[''])

In [8]:
pubs_known_lastauth_country = set(df_scholarly_country['id_select'][(df_scholarly_country['GROUP_lit'].notna()) & (df_scholarly_country['last_author'] == 1)])
len(pubs_known_lastauth_country)

89091

In [10]:
pubs_known_firstauth_country = set(df_scholarly_country['id_select'][(df_scholarly_country['GROUP_lit'].notna()) & (df_scholarly_country['first_author'] == 1)])
len(pubs_known_firstauth_country)

92430

In [12]:
#nr of publication where at least one of the lead authors (first or last) economic group affiliations are known
pubs_known_first_or_last_auth_country = pubs_known_lastauth_country | pubs_known_firstauth_country
len(pubs_known_first_or_last_auth_country)

93616

In [14]:
df_scholarly_country_nn_np = df_scholarly_country[['id_select','GROUP_ori','GROUP_lit','first_author','last_author']].dropna().drop_duplicates()


In [16]:
#how many pubs have sourced at least one sequence from G77?
len(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_ori'] == 'G77']))

24512

In [18]:
pubs_sourced_from_G77 = pd.DataFrame(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_ori'] == 'G77']), columns = ['id_select'])

#pubs_sourced_from_G77.to_csv(path+R'processed_data/pubs_sourced_from_G77.csv')

In [20]:
#how many pubs have have been authored by at least one person from G77?
len(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_lit'] == 'G77']))

16116

In [22]:
len(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_ori'] == 'OECD']))

53685

In [24]:
#how many pubs have have been authored by at least one person from OECD?
len(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_lit'] == 'OECD']))

71524

In [26]:
len(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_ori'] == 'BRICS']))

30806

In [28]:
#how many pubs have have been authored by at least one person from BRICS?
len(set(df_scholarly_country_nn_np['id_select'][df_scholarly_country_nn_np['GROUP_lit'] == 'BRICS']))

28592

In [386]:
# Initialize lists to hold the results
exclusive_oecd_origin = []
exclusive_brics_origin = []
exclusive_g77_origin = []
multiple_origin = []

exclusive_oecd_author = []
exclusive_brics_author = []
exclusive_g77_author = []
multiple_author = []

g77_oecd_1s = []
g77_brics_1s = []
oecd_brics_1s = []
oecd_g77_1s = []
brics_g77_1s = []
brics_oecd_1s = []

g77_oecd_2s = []
g77_brics_2s = []
oecd_brics_2s = []
oecd_g77_2s = []
brics_g77_2s = []
brics_oecd_2s = []

g77_oecds = []
g77_bricss = []
oecd_bricss = []
oecd_g77s = []
brics_g77s = []
brics_oecds = []

g77_oecd_others = []
g77_brics_others = []
oecd_brics_others = []
oecd_g77_others = []
brics_g77_others = []
brics_oecd_others = []
    


# Group the DataFrame by 'id_select'
grouped_df = df_scholarly_country_nn_np.groupby('id_select')

count = 0
for pub, group in grouped_df:
    origin_groups = set(group['GROUP_ori'].dropna())
    first_author_group = set(group['GROUP_lit'][(group['first_author'] == 1)].dropna())
    last_author_group = set(group['GROUP_lit'][(group['last_author'] == 1)].dropna())
    first_last_author_groups = first_author_group.union(last_author_group)
    author_groups = set(group['GROUP_lit'])

    
    #first look into exclusive vs. multiple group types
    if len(origin_groups) == 1: 
        if 'OECD' in origin_groups: 
            exclusive_oecd_origin.append(pub)
        if 'BRICS' in origin_groups: 
            exclusive_brics_origin.append(pub)
        if 'G77' in origin_groups: 
            exclusive_g77_origin.append(pub)
    else:
        multiple_origin.append(pub)

    if len(author_groups) == 1: 
        if 'OECD' in author_groups: 
            exclusive_oecd_author.append(pub)
        if 'BRICS' in author_groups: 
            exclusive_brics_author.append(pub)
        if 'G77' in author_groups: 
            exclusive_g77_author.append(pub)
    else:
        multiple_author.append(pub)


    #general sourcing
    g77_oecd = int('G77' in origin_groups and 'OECD' in author_groups)
    g77_brics = int('G77' in origin_groups and 'BRICS' in author_groups)
    oecd_brics = int('OECD' in origin_groups and 'BRICS' in author_groups)
    oecd_g77 = int('OECD' in origin_groups and 'G77' in author_groups)
    brics_g77 = int('BRICS' in origin_groups and 'G77' in author_groups)
    brics_oecd = int('BRICS' in origin_groups and 'OECD' in author_groups)

    g77_oecd_2 = np.nan
    g77_oecd_1 = np.nan
    g77_oecd_other = np.nan
    g77_brics_2 = np.nan
    g77_brics_1 = np.nan
    g77_brics_other = np.nan
    oecd_brics_2 = np.nan
    oecd_brics_1 = np.nan
    oecd_brics_other = np.nan
    oecd_g77_2 = np.nan
    oecd_g77_1 = np.nan
    oecd_g77_other = np.nan
    brics_g77_2 = np.nan
    brics_g77_1 = np.nan
    brics_g77_other = np.nan
    brics_oecd_2 = np.nan
    brics_oecd_1 = np.nan
    brics_oecd_other = np.nan
    
    
    #label:
    if g77_oecd == 1: 
        if len(author_groups) == 1:
            g77_oecd_2 = 1
            g77_oecd_1 = 0
            g77_oecd_other = 0
        else:
            if 'OECD' in first_last_author_groups and 'G77' not in first_last_author_groups: #def of type one is that G77 are involved but not as leads 
                g77_oecd_1 = 1
                g77_oecd_other = 0
                g77_oecd_2 = 0
            else:
                g77_oecd_1 = 0
                g77_oecd_other = 1
                g77_oecd_2 = 0

    if g77_brics == 1: 
        if len(author_groups) == 1:
            g77_brics_2 = 1
            g77_brics_1 = 0
            g77_brics_other = 0
        else: 
            if 'BRICS' in first_last_author_groups and 'G77' not in first_last_author_groups: #def of type one is that G77 are involved but not as leads 
                g77_brics_1 = 1
                g77_brics_other = 0
                g77_brics_2 = 0
            else:
                g77_brics_1 = 0
                g77_brics_other = 1
                g77_brics_2 = 0

    if oecd_brics == 1: 
        if len(author_groups) == 1:
            oecd_brics_2 = 1
            oecd_brics_1 = 0
            oecd_brics_other = 0
        else: 
            if 'BRICS' in first_last_author_groups and 'OECD' not in first_last_author_groups: #def of type one is that G77 are involved but not as leads 
                oecd_brics_1 = 1
                oecd_brics_other = 0
                oecd_brics_2 = 0
            else:
                oecd_brics_1 = 0
                oecd_brics_other = 1
                oecd_brics_2 = 0

    if oecd_g77 == 1: 
        if len(author_groups) == 1:
            oecd_g77_2 = 1
            oecd_g77_1 = 0
            oecd_g77_other = 0
        else: 
            if 'G77' in first_last_author_groups and 'OECD' not in first_last_author_groups: #def of type one is that G77 are involved but not as leads 
                oecd_g77_1 = 1
                oecd_g77_other = 0
                oecd_g77_2 = 0
            else:
                oecd_g77_1 = 0
                oecd_g77_other = 1
                oecd_g77_2 = 0

    if brics_g77 == 1: 
        if len(author_groups) == 1:
            brics_g77_2 = 1
            brics_g77_1 = 0
            brics_g77_other = 0
        else: 
            if 'G77' in first_last_author_groups and 'BRICS' not in first_last_author_groups: #def of type one is that G77 are involved but not as leads 
                brics_g77_1 = 1
                brics_g77_other = 0
                brics_g77_2 = 0
            else:
                brics_g77_1 = 0
                brics_g77_other = 1
                brics_g77_2 = 0

    if brics_oecd == 1: 
        if len(author_groups) == 1:
            brics_oecd_2 = 1
            brics_oecd_1 = 0
            brics_oecd_other = 0
        else: 
            if 'OECD' in first_last_author_groups and 'BRICS' not in first_last_author_groups: #def of type one is that G77 are involved but not as leads 
                brics_oecd_1 = 1
                brics_oecd_other = 0
                brics_oecd_2 = 0
            else:
                brics_oecd_1 = 0
                brics_oecd_other = 1
                brics_oecd_2 = 0
    
    g77_oecds.append(g77_oecd)
    g77_bricss.append(g77_brics)
    oecd_bricss.append(oecd_brics)
    oecd_g77s.append(oecd_g77)
    brics_g77s.append(brics_g77)
    brics_oecds.append(brics_oecd)
    
    g77_oecd_1s.append(g77_oecd_1)
    g77_brics_1s.append(g77_brics_1)
    oecd_brics_1s.append(oecd_brics_1)
    oecd_g77_1s.append(oecd_g77_1)
    brics_g77_1s.append(brics_g77_1)
    brics_oecd_1s.append(brics_oecd_1)

    g77_oecd_2s.append(g77_oecd_2)
    g77_brics_2s.append(g77_brics_2)
    oecd_brics_2s.append(oecd_brics_2)
    oecd_g77_2s.append(oecd_g77_2)
    brics_g77_2s.append(brics_g77_2)
    brics_oecd_2s.append(brics_oecd_2)

    g77_oecd_others.append(g77_oecd_other)
    g77_brics_others.append(g77_brics_other)
    oecd_brics_others.append(oecd_brics_other)
    oecd_g77_others.append(oecd_g77_other)
    brics_g77_others.append(brics_g77_other)
    brics_oecd_others.append(brics_oecd_other)
    
    count = count+1
    #if count == 15: 
    #    break


# Create DataFrame directly from the group key (id_select) and the calculated lists
df_extract = pd.DataFrame({
    'id_select': grouped_df['id_select'].first().index,
    'extractivism_g77_oecd_1': g77_oecd_1s,
    'extractivism_g77_brics_1': g77_brics_1s,
    'extractivism_oecd_brics_1': oecd_brics_1s,
    'extractivism_oecd_g77_1': oecd_g77_1s,
    'extractivism_brics_g77_1': brics_g77_1s,
    'extractivism_brics_oecd_1': brics_oecd_1s,
    'extractivism_g77_oecd_2': g77_oecd_2s,
    'extractivism_g77_brics_2': g77_brics_2s,
    'extractivism_oecd_brics_2': oecd_brics_2s,
    'extractivism_oecd_g77_2': oecd_g77_2s,
    'extractivism_brics_g77_2': brics_g77_2s,
    'extractivism_brics_oecd_2': brics_oecd_2s,
    'g77_oecd': g77_oecds,
    'g77_brics': g77_bricss,
    'oecd_brics': oecd_bricss,
    'oecd_g77': oecd_g77s,
    'brics_g77': brics_g77s,
    'brics_oecd': brics_oecds,
    'other_g77_oecd': g77_oecd_others,
    'other_g77_brics': g77_brics_others,
    'other_oecd_brics': oecd_brics_others,
    'other_oecd_g77': oecd_g77_others,
    'other_brics_g77': brics_g77_others,
    'other_brics_oecd': brics_oecd_others
})


In [ ]:
#how many extractivisms

In [118]:
#nr of unclear leadauthors (so they are missing type 1)
len(df_extract[df_extract['extractivism_g77_oecd_1'].isna()])

88861

In [276]:
len(df_extract[df_extract['brics_oecd'] == 1])

12889

In [388]:
len(df_extract[df_extract['other_brics_oecd'] == 1])

5598

In [280]:
len(df_extract[df_extract['extractivism_brics_oecd_1'] == 1])

2353

In [282]:
len(df_extract[df_extract['extractivism_brics_oecd_2'] == 1])

4938

In [284]:
len(df_extract[df_extract['g77_oecd'] == 1])

18907

In [286]:
len(df_extract[df_extract['extractivism_g77_oecd_2'] == 1])

8647

In [288]:
len(df_extract[df_extract['extractivism_g77_oecd_1'] == 1])

5410

In [390]:
len(df_extract[df_extract['other_g77_oecd'] == 1])

4850

In [292]:
len(df_extract[df_extract['oecd_brics'] == 1])

4189

In [294]:
len(df_extract[df_extract['extractivism_oecd_brics_2'] == 1])

896

In [296]:
len(df_extract[df_extract['extractivism_oecd_brics_1'] == 1])

720

In [392]:
len(df_extract[df_extract['other_oecd_brics'] == 1])

2573

In [ ]:
#exclusive origins

In [64]:
len(exclusive_oecd_origin)

44339

In [66]:
len(exclusive_brics_origin)

23699

In [68]:
len(exclusive_g77_origin)

15925

In [80]:
len(multiple_origin)

10809

In [82]:
len(exclusive_oecd_author)

52467

In [84]:
len(exclusive_brics_author)

17765

In [86]:
len(exclusive_g77_author)

4549

In [88]:
len(multiple_author)

19991

In [424]:
df_extract.to_csv(path+R'processed_data\extractivism.csv')

In [394]:
df_extract

,id_select,extractivism_g77_oecd_1,extractivism_g77_brics_1,extractivism_oecd_brics_1,extractivism_oecd_g77_1,extractivism_brics_g77_1,extractivism_brics_oecd_1,extractivism_g77_oecd_2,extractivism_g77_brics_2,extractivism_oecd_brics_2,...,oecd_brics,oecd_g77,brics_g77,brics_oecd,other_g77_oecd,other_g77_brics,other_oecd_brics,other_oecd_g77,other_brics_g77,other_brics_oecd
0,10.1002/jpln.201100382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,10.1002/jsfa.8281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,10.1002/mrd.22210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,10.1006/bojl.2001.0485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,10.1007/s001220050005,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94767,pmc93007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
94768,pmc93245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
94769,pmc93301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
94770,pmc96788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [398]:
df_extract[df_extract['g77_oecd']== 1]

,id_select,extractivism_g77_oecd_1,extractivism_g77_brics_1,extractivism_oecd_brics_1,extractivism_oecd_g77_1,extractivism_brics_g77_1,extractivism_brics_oecd_1,extractivism_g77_oecd_2,extractivism_g77_brics_2,extractivism_oecd_brics_2,...,oecd_brics,oecd_g77,brics_g77,brics_oecd,other_g77_oecd,other_g77_brics,other_oecd_brics,other_oecd_g77,other_brics_g77,other_brics_oecd
7,10.1007/s00203-012-0860-8,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
9,10.1007/s00227-014-2396-8,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
12,10.1007/s00248-012-0110-2,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
32,10.1007/s00374-012-0705-2,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
37,10.1007/s00436-011-2612-0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94737,pmc6628216,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
94738,pmc6628235,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
94741,pmc6629698,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
94745,pmc6632193,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN


In [402]:
df_extract[df_extract['extractivism_g77_oecd_2'].notna()]

,id_select,extractivism_g77_oecd_1,extractivism_g77_brics_1,extractivism_oecd_brics_1,extractivism_oecd_g77_1,extractivism_brics_g77_1,extractivism_brics_oecd_1,extractivism_g77_oecd_2,extractivism_g77_brics_2,extractivism_oecd_brics_2,...,oecd_brics,oecd_g77,brics_g77,brics_oecd,other_g77_oecd,other_g77_brics,other_oecd_brics,other_oecd_g77,other_brics_g77,other_brics_oecd
7,10.1007/s00203-012-0860-8,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
9,10.1007/s00227-014-2396-8,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
12,10.1007/s00248-012-0110-2,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
32,10.1007/s00374-012-0705-2,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
37,10.1007/s00436-011-2612-0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94737,pmc6628216,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
94738,pmc6628235,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
94741,pmc6629698,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
94745,pmc6632193,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN


In [416]:
df_extract[df_extract['other_g77_oecd']== 1]

,id_select,extractivism_g77_oecd_1,extractivism_g77_brics_1,extractivism_oecd_brics_1,extractivism_oecd_g77_1,extractivism_brics_g77_1,extractivism_brics_oecd_1,extractivism_g77_oecd_2,extractivism_g77_brics_2,extractivism_oecd_brics_2,...,oecd_brics,oecd_g77,brics_g77,brics_oecd,other_g77_oecd,other_g77_brics,other_oecd_brics,other_oecd_g77,other_brics_g77,other_brics_oecd
32,10.1007/s00374-012-0705-2,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
65,10.1007/s00606-008-0026-y,0.0,NaN,NaN,0.0,1.0,1.0,0.0,NaN,NaN,...,0,1,1,1,1.0,NaN,NaN,1.0,0.0,0.0
66,10.1007/s00606-008-0053-8,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,0,1,0,0,1.0,NaN,NaN,1.0,NaN,NaN
104,10.1007/s00606-017-1450-7,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
110,10.1007/s10096-010-1071-3,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94699,pmc6620521,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
94704,pmc6624209,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
94707,pmc6624733,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN
94745,pmc6632193,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN


In [372]:
df_extract[df_extract['extractivism_g77_oecd_2'] == 1]

,id_select,extractivism_g77_oecd_1,extractivism_g77_brics_1,extractivism_oecd_brics_1,extractivism_oecd_g77_1,extractivism_brics_g77_1,extractivism_brics_oecd_1,extractivism_g77_oecd_2,extractivism_g77_brics_2,extractivism_oecd_brics_2,...,oecd_brics,oecd_g77,brics_g77,brics_oecd,g77_oecd_others,g77_brics_others,oecd_brics_others,oecd_g77_others,brics_g77_others,brics_oecd_others
7,10.1007/s00203-012-0860-8,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
12,10.1007/s00248-012-0110-2,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
37,10.1007/s00436-011-2612-0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48,10.1007/s00606-006-0474-1,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
52,10.1007/s00606-007-0529-y,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94615,pmc6602244,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
94641,pmc6609226,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
94647,pmc6609996,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
94649,pmc6610071,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [370]:
df_extract[df_extract['extractivism_g77_oecd_1'] == 1]

,id_select,extractivism_g77_oecd_1,extractivism_g77_brics_1,extractivism_oecd_brics_1,extractivism_oecd_g77_1,extractivism_brics_g77_1,extractivism_brics_oecd_1,extractivism_g77_oecd_2,extractivism_g77_brics_2,extractivism_oecd_brics_2,...,oecd_brics,oecd_g77,brics_g77,brics_oecd,g77_oecd_others,g77_brics_others,oecd_brics_others,oecd_g77_others,brics_g77_others,brics_oecd_others
9,10.1007/s00227-014-2396-8,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
45,10.1007/s00572-013-0549-0,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
49,10.1007/s00606-006-0488-8,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
76,10.1007/s00606-009-0177-5,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,0,1,0,0,NaN,NaN,NaN,1.0,NaN,NaN
81,10.1007/s00606-009-0213-5,1.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,1,0,0,1,NaN,NaN,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94716,pmc6626051,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,0,1,0,0,NaN,NaN,NaN,1.0,NaN,NaN
94735,pmc6627985,1.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,...,1,0,0,0,NaN,1.0,1.0,NaN,NaN,NaN
94737,pmc6628216,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
94738,pmc6628235,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Trash
# Initialize lists to hold the results
exclusive_oecd_origin = []
exclusive_brics_origin = []
exclusive_g77_origin = []
multiple_origin = []

exclusive_oecd_author = []
exclusive_brics_author = []
exclusive_g77_author = []
multiple_author = []

g77_oecd_1s = []
g77_brics_1s = []
oecd_brics_1s = []
oecd_g77_1s = []
brics_g77_1s = []
brics_oecd_1s = []

g77_oecd_2s = []
g77_brics_2s = []
oecd_brics_2s = []
oecd_g77_2s = []
brics_g77_2s = []
brics_oecd_2s = []

#coorp_g77_oecd = []
#coorp_g77_brics = []
    


# Group the DataFrame by 'id_select'
grouped_df = df_scholarly_country_nn_np.groupby('id_select')

count = 0
for pub, group in grouped_df:
    origin_groups = set(group['GROUP_ori'].dropna())
    first_author_group = set(group['GROUP_lit'][(group['first_author'] == 1)].dropna())
    last_author_group = set(group['GROUP_lit'][(group['last_author'] == 1)].dropna())
    first_last_author_groups = first_author_group.union(last_author_group)
    author_groups = set(group['GROUP_lit'])

    
    #first look into exclusive vs. multiple group types
    if len(origin_groups) == 1: 
        if 'OECD' in origin_groups: 
            exclusive_oecd_origin.append(pub)
        if 'BRICS' in origin_groups: 
            exclusive_brics_origin.append(pub)
        if 'G77' in origin_groups: 
            exclusive_g77_origin.append(pub)
    else:
        multiple_origin.append(pub)

    if len(author_groups) == 1: 
        if 'OECD' in author_groups: 
            exclusive_oecd_author.append(pub)
        if 'BRICS' in author_groups: 
            exclusive_brics_author.append(pub)
        if 'G77' in author_groups: 
            exclusive_g77_author.append(pub)
    else:
        multiple_author.append(pub)

    
    #label everything regardless of whether first and last are known
    g77_oecd_2 = int('G77' in origin_groups and len(author_groups) == 1 and 'OECD' in author_groups)
    g77_brics_2 = int('G77' in origin_groups and len(author_groups) == 1 and 'BRICS' in author_groups)
    oecd_brics_2 = int('OECD' in origin_groups and len(author_groups) == 1 and 'BRICS' in author_groups)
    oecd_g77_2 = int('OECD' in origin_groups and len(author_groups) == 1 and 'G77' in author_groups)
    brics_g77_2 = int('BRICS' in origin_groups and len(author_groups) == 1 and 'G77' in author_groups)
    brics_oecd_2 = int('BRICS' in origin_groups and len(author_groups) == 1 and 'OECD' in author_groups)

    
    #now classify extractivism type    
    #if first or last author known, label
    if len(first_last_author_groups) == 1: 
        g77_oecd_1 = 0
        g77_brics_1 = 0
        oecd_brics_1 = 0
        oecd_g77_1 = 0
        brics_g77_1 = 0
        brics_oecd_1 = 0
        
        if g77_oecd_2 == 0:
            g77_oecd_1 = int('G77' in origin_groups and 'OECD' in first_last_author_groups)
        if g77_brics_2 == 0:
            g77_brics_1 = int('G77' in origin_groups and 'BRICS' in first_last_author_groups)
        if oecd_brics_2 == 0:
            oecd_brics_1= int('OECD' in origin_groups and 'BRICS' in first_last_author_groups)
        if oecd_g77_2 == 0:
            oecd_g77_1 = int('OECD' in origin_groups and 'G77' in first_last_author_groups)
        if brics_g77_2 == 0:
            brics_g77_1 = int('BRICS' in origin_groups and 'G77' in first_last_author_groups)
        if brics_oecd_2 == 0:
            brics_oecd_1 = int('BRICS' in origin_groups and 'OECD' in first_last_author_groups)
        
    
    #else label as unknown
    else: 
        g77_oecd_1 = np.nan
        g77_brics_1 = np.nan
        oecd_brics_1 = np.nan
        oecd_g77_1 = np.nan
        brics_g77_1 = np.nan
        brics_oecd_1 = np.nan
        print(pub)
    
    #if not pd.isna(g77_oecd_1):
    #    coorp_oecd = int('G77' in author_groups and 'OECD' in author_groups and g77_oecd_1 == 0 and g77_oecd_2 == 0)
    #    coorp_brics = int('G77' in author_groups and 'BRICS' in author_groups and g77_brics_1 == 0 and g77_brics_2 == 0)
    #else:
    #    coorp_g77_oecd   = np.nan
    #    coorp_g77_brics  = np.nan
    
    
    g77_oecd_1s.append(g77_oecd_1)
    g77_brics_1s.append(g77_brics_1)
    oecd_brics_1s.append(oecd_brics_1)
    oecd_g77_1s.append(oecd_g77_1)
    brics_g77_1s.append(brics_g77_1)
    brics_oecd_1s.append(brics_oecd_1)

    g77_oecd_2s.append(g77_oecd_2)
    g77_brics_2s.append(g77_brics_2)
    oecd_brics_2s.append(oecd_brics_2)
    oecd_g77_2s.append(oecd_g77_2)
    brics_g77_2s.append(brics_g77_2)
    brics_oecd_2s.append(brics_oecd_2)
    
    
    count = count+1
    #if count == 15: 
    #    break


# Create DataFrame directly from the group key (id_select) and the calculated lists
df_extract = pd.DataFrame({
    'id_select': grouped_df['id_select'].first().index,
    'extractivism_g77_oecd_1': g77_oecd_1s,
    'extractivism_g77_brics_1': g77_brics_1s,
    'extractivism_oecd_brics_1': oecd_brics_1s,
    'extractivism_oecd_g77_1': oecd_g77_1s,
    'extractivism_brics_g77_1': brics_g77_1s,
    'extractivism_brics_oecd_1': brics_oecd_1s,
    'extractivism_g77_oecd_2': g77_oecd_2s,
    'extractivism_g77_brics_2': g77_brics_2s,
    'extractivism_oecd_brics_2': oecd_brics_2s,
    'extractivism_oecd_g77_2': oecd_g77_2s,
    'extractivism_brics_g77_2': brics_g77_2s,
    'extractivism_brics_oecd_2': brics_oecd_2s
})
